# Scraper of public procurement processes

The webpage that has the data on all the contracts for medicine, which can be found [here](https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1#), only allows searches in 6 month periods, and has a captcha. This scraper allows to download all of the purchasing processes using Playwright and bs4, but filling the date and captcha mannually.

Purchasing processes were launched in 2022 and 2023; there are no processes for 2024 and 2025.

In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import asyncio

In [2]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1#")

<Response url='https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1' request=<Request url='https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1' method='GET'>>

In [3]:
all_rows = []

semesters = [
    ("First 2022", "Waiting for dates and captcha manually"),
    ("Second 2022", "Waiting for dates and captcha manually"),
    ("First 2023", "Waiting for dates and captcha manually"),
    ("Second 2023", "Waiting for dates and captcha manually"),
]

for label, instruction in semesters:
    print(f"Scraping {label} — {instruction}")
    input("---- Press enter when completed")

    offset = 0

    while True:
        try:
            await page.wait_for_selector("table", timeout=10000)
            html = await page.content()
            soup = BeautifulSoup(html, "lxml")

            rows = soup.select("table tr")[1:]  # Skip header
            for row in rows:
                cols = row.find_all("td")
                if not cols or len(cols) < 7:
                    continue
                try:
                    codigo = cols[0].text.strip()
                    detalle_url = cols[0].find("a")["href"] if cols[0].find("a") else None
                    entidad = cols[1].text.strip()
                    objeto = cols[2].text.strip()
                    estado = cols[3].text.strip()
                    ubicacion = cols[4].text.strip()
                    presupuesto = cols[5].text.strip()
                    fecha_publicacion = cols[6].text.strip()

                    all_rows.append({
                        "codigo": codigo,
                        "detalle_url": detalle_url,
                        "entidad": entidad,
                        "objeto": objeto,
                        "estado": estado,
                        "ubicacion": ubicacion,
                        "presupuesto": presupuesto,
                        "fecha_publicacion": fecha_publicacion,
                        "semestre": label
                    })
                except Exception as e:
                    print("Error parsing row:", e)

            try:
                siguiente_button = await page.query_selector("a:has-text('Siguiente')")
                if not siguiente_button:
                    print("No 'Siguiente' button found — ending this semester")
                    break

                offset += 20
                await asyncio.sleep(2)
                await page.evaluate(f"presentarProcesos({offset})")

            except Exception as e:
                print("Error checking or clicking 'Siguiente':", e)
                break

        except Exception as e:
            print("Unexpected error while scraping:", e)
            break

    print(f"Finished scraping for: {label}")
    await asyncio.sleep(5)

Scraping First 2022 — Waiting for dates and captcha manually


---- Press enter when completed 


No 'Siguiente' button found — ending this semester
Finished scraping for: First 2022
Scraping Second 2022 — Waiting for dates and captcha manually


---- Press enter when completed 


No 'Siguiente' button found — ending this semester
Finished scraping for: Second 2022
Scraping First 2023 — Waiting for dates and captcha manually


---- Press enter when completed 


No 'Siguiente' button found — ending this semester
Finished scraping for: First 2023
Scraping Second 2023 — Waiting for dates and captcha manually


---- Press enter when completed 


No 'Siguiente' button found — ending this semester
Finished scraping for: Second 2023


In [4]:
df = pd.DataFrame(all_rows)
len(df)

2692

In [5]:
df.head(10)

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion,semestre
0,,None,Entidad Contratante,Buscar Entidad,,Buscar Entidad,Buscar Entidad,Buscar Entidad,First 2022
1,,None,Por Fechas de Publicación (*),Desde: \n Hasta:,Desde:,,,Hasta:,First 2022
2,,None,,,,,,,First 2022
3,,None,,,,,,,First 2022
4,,None,,,,,,,First 2022
5,Código,None,Entidad Contratante,Objeto del Proceso,Estado del Proceso,Provincia/Cantón,Presupuesto Referencial Unitario(sin iva),Fecha de Publicación,First 2022
6,SICM-499-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,Desierto,PICHINCHA / QUITO,$62.31600,2022-06-23 08:00:00,First 2022
7,SICM-514-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: PREDNISOLONA - FORMA FARMACÉUTICA: SÓLIDO...,Desierto,PICHINCHA / QUITO,$0.06000,2022-06-23 08:00:00,First 2022
8,SICM-500-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Adjudicado oferente ganador,PICHINCHA / QUITO,$5.00000,2022-06-23 08:00:00,First 2022
9,SICM-515-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: PROGESTERONA - FORMA FARMACÉUTICA: SÓLIDO...,Desierto,PICHINCHA / QUITO,$0.23000,2022-06-23 08:00:00,First 2022


In [6]:
df["detalle_url"].isna().value_counts()

detalle_url
True     1706
False     986
Name: count, dtype: int64

In [7]:
clean_df = df[df["detalle_url"].notna()]
len(clean_df)

986

In [8]:
clean_df.head()

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion,semestre
6,SICM-499-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,Desierto,PICHINCHA / QUITO,$62.31600,2022-06-23 08:00:00,First 2022
7,SICM-514-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: PREDNISOLONA - FORMA FARMACÉUTICA: SÓLIDO...,Desierto,PICHINCHA / QUITO,$0.06000,2022-06-23 08:00:00,First 2022
8,SICM-500-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Adjudicado oferente ganador,PICHINCHA / QUITO,$5.00000,2022-06-23 08:00:00,First 2022
9,SICM-515-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: PROGESTERONA - FORMA FARMACÉUTICA: SÓLIDO...,Desierto,PICHINCHA / QUITO,$0.23000,2022-06-23 08:00:00,First 2022
10,SICM-516-2022,informacionProcesoContratacion2.cpe?idSoliComp...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: RITUXIMAB - FORMA FARMACÉUTICA: LÍQUIDO P...,Desierto,PICHINCHA / QUITO,$1406.37000,2022-06-23 08:00:00,First 2022


In [9]:
base_url = "https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/"
clean_df["detalle_url"] = base_url + clean_df["detalle_url"].astype(str)

/var/folders/g1/ld0klj_s7bxf96phwdd6v4t80000gn/T/ipykernel_4965/387573897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df["detalle_url"] = base_url + clean_df["detalle_url"].astype(str)


In [10]:
clean_df.head(1)

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion,semestre
6,SICM-499-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,Desierto,PICHINCHA / QUITO,$62.31600,2022-06-23 08:00:00,First 2022


In [12]:
len(clean_df)

986

In [11]:
clean_df.to_csv("compras_medicinas_2022-2023.csv", index=False)